[![AWS SDK for pandas](_static/logo.png "AWS SDK for pandas")](https://github.com/aws/aws-sdk-pandas)

# 19 - Amazon Athena Cache

[awswrangler](https://github.com/aws/aws-sdk-pandas) has a cache strategy that is disabled by default and can be enabled by passing `max_cache_seconds` bigger than 0 as part of the `athena_cache_settings` parameter. This cache strategy for Amazon Athena can help you to **decrease query times and costs**.

When calling `read_sql_query`, instead of just running the query, we now can verify if the query has been run before. If so, and this last run was within `max_cache_seconds` (a new parameter to `read_sql_query`), we return the same results as last time if they are still available in S3. We have seen this increase performance more than 100x, but the potential is pretty much infinite.

The detailed approach is:
- When `read_sql_query` is called with `max_cache_seconds > 0` (it defaults to 0), we check for the last queries run by the same workgroup (the most we can get without pagination).
- By default it will check the last 50 queries, but you can customize it through the `max_cache_query_inspections` argument.
- We then sort those queries based on CompletionDateTime, descending
- For each of those queries, we check if their CompletionDateTime is still within the `max_cache_seconds` window. If so, we check if the query string is the same as now (with some smart heuristics to guarantee coverage over both `ctas_approach`es). If they are the same, we check if the last one's results are still on S3, and then return them instead of re-running the query.
- During the whole cache resolution phase, if there is anything wrong, the logic falls back to the usual `read_sql_query` path.

*P.S. The `cache scope is bounded for the current workgroup`, so you will be able to reuse queries results from others colleagues running in the same environment.*

In [1]:
import awswrangler as wr

2023-04-15 01:25:39,171	INFO worker.py:1553 -- Started a local Ray instance.


## Enter your bucket name:

In [2]:
import getpass
bucket = getpass.getpass()
path = f"s3://{bucket}/data/"

## Checking/Creating Glue Catalog Databases

In [3]:
if "awswrangler_test" not in wr.catalog.databases().values:
    wr.catalog.create_database("awswrangler_test")

### Creating a Parquet Table from the NOAA's CSV files

[Reference](https://registry.opendata.aws/noaa-ghcn/)

In [9]:
cols = ["id", "dt", "element", "value", "m_flag", "q_flag", "s_flag", "obs_time"]

df = wr.s3.read_csv(
    path="s3://noaa-ghcn-pds/csv/by_year/1880.csv",
    names=cols,
    parse_dates=["dt", "obs_time"])

df

Read progress: 100%|██████████| 1/1 [00:00<00:00, 634.25it/s]


,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,ID,DATE,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
1,ASN00023000,18400101,PRCP,0,NaN,NaN,a,NaN
2,BE000006447,18400101,TMAX,111,NaN,NaN,I,NaN
3,BE000006447,18400101,TMIN,43,NaN,NaN,I,NaN
4,EZE00100082,18400101,TMAX,-4,NaN,NaN,E,NaN
...,...,...,...,...,...,...,...,...
8663,SWE00139148,18401231,PRCP,0,NaN,NaN,E,NaN
8664,UK000047811,18401231,PRCP,1,NaN,NaN,E,NaN
8665,UK000056225,18401231,TMAX,66,NaN,NaN,E,NaN
8666,UK000056225,18401231,TMIN,25,NaN,NaN,E,NaN


In [10]:
wr.s3.to_parquet(
    df=df,
    path=path,
    dataset=True,
    mode="overwrite",
    database="awswrangler_test",
    table="noaa"
)

Write Progress: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


{'paths': ['s3://ibmsample/data/e4296c909be94cfebe80b381163c3892_000000.snappy.parquet'],
 'partitions_values': {}}

In [11]:
wr.catalog.table(database="awswrangler_test", table="noaa")

,Column Name,Type,Partition,Comment
0,id,string,False,
1,dt,string,False,
2,element,string,False,
3,value,string,False,
4,m_flag,string,False,
5,q_flag,string,False,
6,s_flag,string,False,
7,obs_time,string,False,


## The test query

The more computational resources the query needs, the more the cache will help you. That's why we're doing it using this long running query.

In [12]:
query = """
SELECT
    n1.element,
    count(1) as cnt
FROM
    noaa n1
JOIN
    noaa n2
ON
    n1.id = n2.id
GROUP BY
    n1.element
"""

## First execution...

In [13]:
%%time

wr.athena.read_sql_query(query, database="awswrangler_test")

Read progress: 100%|██████████| 1/1 [00:00<00:00,  6.85it/s]


CPU times: user 1.44 s, sys: 139 ms, total: 1.58 s
Wall time: 6.83 s


,element,cnt
0,SNOW,175812
1,TMIN,2568018
2,ELEMENT,1
3,PRCP,2405759
4,TMAX,2032194


## Second execution with **CACHE** (400x faster)

In [16]:
%%time

wr.athena.read_sql_query(query, database="awswrangler_test", athena_cache_settings={"max_cache_seconds":900})

Read progress: 100%|██████████| 1/1 [00:00<00:00,  7.99it/s]

CPU times: user 437 ms, sys: 27.1 ms, total: 464 ms
Wall time: 3.13 s


,element,cnt
0,SNOW,175812
1,TMIN,2568018
2,ELEMENT,1
3,PRCP,2405759
4,TMAX,2032194


## Allowing awswrangler to inspect up to 500 historical queries to find same result to reuse.

In [17]:
%%time

wr.athena.read_sql_query(query, database="awswrangler_test", athena_cache_settings={"max_cache_seconds": 900, "max_cache_query_inspections": 500})

Read progress: 100%|██████████| 1/1 [00:00<00:00,  2.69it/s]

CPU times: user 633 ms, sys: 27.6 ms, total: 660 ms
Wall time: 3.44 s


,element,cnt
0,SNOW,175812
1,TMIN,2568018
2,ELEMENT,1
3,PRCP,2405759
4,TMAX,2032194


## Cleaning Up S3

In [18]:
wr.s3.delete_objects(path)

## Delete table

In [19]:
wr.catalog.delete_table_if_exists(database="awswrangler_test", table="noaa")

True

## Delete Database

In [20]:
wr.catalog.delete_database('awswrangler_test')